# 06 · Core Attacks

Compute loss/confidence, Win-k, and label-free scores for member vs non-member cohorts.

In [ ]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path(os.getcwd()).resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")


In [ ]:
import numpy as np

from src.attacks.loss_confidence import score_examples
from src.attacks.win_k_min_k import aggregate_features
from src.attacks.label_free import CalibrationPrompt, summarize_outputs

rng = np.random.default_rng(1)
BATCH = 32
TOKENS = 16
nll = rng.random((BATCH, TOKENS))
entropy = rng.random((BATCH, TOKENS))
max_prob = rng.random((BATCH, TOKENS))
win_dict = {f'win@{k}': rng.random((BATCH, TOKENS)) > 0.5 for k in (1, 5, 10, 20)}

loss_features = score_examples(nll, entropy, max_prob)
win_features = aggregate_features(win_dict, nll, worst_percents=[0.05, 0.10])

prompt = CalibrationPrompt()
responses = ['Synthetic response' for _ in range(BATCH)]
label_free_scores = summarize_outputs([prompt.format('context')]*BATCH, responses)

print('Loss features keys:', loss_features.keys())
print('Win-k feature sample:', {k: v[:3] for k, v in win_features.items()})
print('Label-free scores sample:', label_free_scores[:5])

📝 **TODO:** Join these features with actual member/non-member labels and persist to `reports/features_slice_t.parquet`.